This code runs the analyses for Intrator's study "Effects of Health and Economic Crises on Healthcare Spending in the United States"

In [1]:
## package imports
import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from stargazer.stargazer import Stargazer

In [2]:
## data imports

## OECD datasets
main = pd.read_csv('clean dataset.csv').set_index('year')

## CDC datasets
aids = pd.read_csv('cleaned aids data - 1970-2015.csv').set_index('year')
aids = aids[(aids.index >= 1970) & (aids.index <= 2013)] ## getting rid of years outside of period of analysis
opioid = pd.read_csv('cleaned opioid data - 1999-2015.csv').set_index('year')
opioid = opioid[(opioid.index >= 1970) & (opioid.index <= 2013)]

## BLS datasets
cpi = pd.read_csv('clean medical cpi.csv').set_index('year')
cpi = cpi[(cpi.index >=1970) & (cpi.index <= 2013)]

# Summary Statistics for All Variables

In [3]:
## starting with an empty list to which all the relevant values will be appended
summary = []

## appending lists with relevant summary statistics and variable names for the main dataframe
for col in main.columns:
    summary.append([
        col, len(main[col]), np.mean(main[col]), np.std(main[col]), np.min(main[col]), np.max(main[col])
    ])

## appending a list with the relevant values for the medical care CPI data
summary.append([
    'CPI', len(cpi), np.mean(cpi.cpi), np.std(cpi.cpi), np.min(cpi.cpi), np.max(cpi.cpi)
])
    
## appending a list with the relevant values for the opioid crude death rate
summary.append([
    'opioid_crude', len(opioid), np.mean(opioid.opioid_crude), np.std(opioid.opioid_crude), np.min(opioid.opioid_crude),
    np.max(opioid.opioid_crude)
])

## appending a list with the relevant values for the HIV/AIDS crude death rate
summary.append([
    'aids_crude', 2013-1979, np.mean(aids.aids_crude), np.std(aids.aids_crude), np.min(aids.aids_crude),
    np.max(aids.aids_crude)
])

## initializing a list for the correct column names for the summary statistics dataframe that will be presented
summary_cols = ['variable', 'n', 'mean', 'standard deviation', 'min', 'max']

## turning the summary statistics list-of-lists into a dataframe, setting its columns, and then setting its index to be
## the variable names
summary = pd.DataFrame(summary, columns = summary_cols).set_index('variable')

## sending the summary statistics dataframe to a .csv file
summary.to_csv('summary stats.csv')

# Final Data Cleaning and Combining of Each Dataset

## creating dataframe for the baseline independent variables (OECD independents)

In [4]:
## copying relevant columns
base_indep = main[main.columns[4:]]

## taking the natural logarithm and first difference of all variables, then removing the null values
base_indep = np.log(base_indep).diff().dropna()

## adding a constant using statsmodels
base_indep = sm.add_constant(base_indep)

## adding a linear time trend based on the year
base_indep['t'] = base_indep.index - 1970

## creating dataframe for health crises variables

In [5]:
## creating a copy of the aids dataframe
aids_prep = aids.copy()

## taking the natural logarithm of the crude death rate and the interaction between crude death rate and reliable years
## (will result in some negative infinity values because we are taking the natural logarithm of 0 on multiple occasions,
## but we will replace those values with 0 as they are supposed to be 0 anyway for the regressions)
aids_prep[['aids_crude', 'aids_rel_inter']] = np.log(aids_prep[['aids_crude', 'aids_rel_inter']])

## replacing -inf with 0s as described above
aids_prep[['aids_crude', 'aids_rel_inter']] = aids_prep[['aids_crude', 'aids_rel_inter']].replace(np.float('-inf'), 0)

## taking first difference of now logged variables of interest (not the dummy variable for reliable data)
aids_prep[['aids_crude', 'aids_rel_inter']] = aids_prep[['aids_crude', 'aids_rel_inter']].diff()

## removing null values from the dataset
aids_prep = aids_prep.dropna()

## reordering the columns so the interaction is the last column
aids_prep = aids_prep[['aids_crude', 'aids_rel_inter', 'reliable']]

'''Do not worry about the error message involving the natural logarithms, that is simply a numpy issue that is fixed
when those negative infinity values are replaced with 0s in this cell.'''

<ipython-input-5-aa8f85d96717>:7: RuntimeWarning: divide by zero encountered in log
  aids_prep[['aids_crude', 'aids_rel_inter']] = np.log(aids_prep[['aids_crude', 'aids_rel_inter']])


'Do not worry about the error message involving the natural logarithms, that is simply a numpy issue that is fixed\nwhen those negative infinity values are replaced with 0s in this cell.'

In [6]:
## creating a copy of the opioid data
op_prep = opioid.copy()

## taking the natural logarithm of the opioid variable
op_prep = np.log(op_prep)

## creating a dataframe of 0s to be appended to the beginning of the opioid data to account for years where opioid deaths
## went unnoticed and/or relatively unmeasured
zeros = pd.DataFrame(
    {'year': list(range(1970,1999)), 'opioid_crude': [0]*len(range(1970,1999))}
    ).set_index('year')

## appending the 0s and the natural logarithm of the opioid data
op_prep = zeros.append(op_prep)

## taking the first difference of the opioid data and then removing the null values
op_prep = op_prep.diff().dropna()

In [7]:
## combining the prepped opioid and aids data into a single dataframe
health_crises = op_prep.join(aids_prep)

## removing values not in the period of analysis (1970-2013, although we start at 1971 here because of first differencing)
health_crises = health_crises[health_crises.index <= 2013]

In [8]:
## creating the dataframe for independent variables which include health crises
crisis_indep = base_indep.join(health_crises)

## creating dataframes for the dependent variables
combining the spending variables with CPI, then dividing to obtain quantity measures

In [9]:
## creating a dataframe which is just a copy of all the spending metrics
spending = main[main.columns[:4]].copy()

## creating a list of renamed columns where we drop the '_USD_CAP' at the end for easier reading (all spending metrics
## are already in US dollars per capita)
spending_names = [col[:-8] for col in spending.columns]

## renaming the columns of the spending data to match these values
spending.columns = spending_names

In [10]:
## joining the medical care CPI measure to spending to more easily calculate the quantities
spending = cpi.join(spending).dropna()

In [11]:
## creating the quantity measures by dividing each spending metric by the CPI
for col in spending_names:
    spending[col+'_q'] = spending[col] / spending['cpi']

In [12]:
## creating multiple dataframes which will act as the dependent variables for each part of the analysis

## creating the baseline dependent variables, which will include total spending, total quantity, and CPI
base_dep = spending[['TOT', 'cpi', 'TOT_q']].copy()

## creating the segmented dependent variables, which will include the spending and quantities for OOPEXP, COMPULSORY,
## & VOLUNTARY
segmented_dep = spending[['OOPEXP', 'OOPEXP_q', 'VOLUNTARY', 'VOLUNTARY_q', 'COMPULSORY', 'COMPULSORY_q']].copy()

In [13]:
## taking the natural logarithm, first difference, and removing null values for each of these dependent measures here
base_dep = np.log(base_dep).diff().dropna()
segmented_dep = np.log(segmented_dep).diff().dropna()

# Running the Regressions on the Data

Using statsmodels to run the regressions and then using Stargazer to combine the results into a pretty format that can be exported nicely

In [14]:
## initializing an empty list to store all the regressions for the baseline models
base_regs = []

## running a loop to regress the base_dep variables on the base_indep variables, then fitting the regression using 'HC1'
## estimated robust standard errors
for col in base_dep:
    base_regs.append(sm.OLS(base_dep[col], base_indep).fit(cov_type='HC1'))
    
## combining the results into a pretty stargazer format
base_results = Stargazer(base_regs)
base_results.custom_columns(base_dep.columns.tolist(), [1]*len(base_dep.columns))
base_results.show_model_numbers(False)
base_results.covariate_order(base_indep.columns.tolist())
base_results.significance_levels([0.05, 0.01, 0.001])
base_results

In [15]:
## initializing an empty list to store all the regressions for the crisis models
crisis_regs = []

## running a loop to regress the base_dep variables on the crisis_indep variables, then fitting the regression using 'HC1'
## estimated robust standard errors
for col in base_dep:
    crisis_regs.append(sm.OLS(base_dep[col], crisis_indep).fit(cov_type='HC1'))
    
## combining the results into a pretty stargazer format
crisis_results = Stargazer(crisis_regs)
crisis_results.custom_columns(base_dep.columns.tolist(), [1]*len(base_dep.columns))
crisis_results.show_model_numbers(False)
crisis_results.covariate_order(crisis_indep.columns.tolist())
crisis_results.significance_levels([0.05, 0.01, 0.001])
crisis_results

In [16]:
## initializing an empty list to store all the regressions for the segmented analyses
segmented_regs = []

## running a loop to regress the base_dep variables on the base_indep variables, then fitting the regression using 'HC1'
## estimated robust standard errors
for col in segmented_dep:
    segmented_regs.append(sm.OLS(segmented_dep[col], crisis_indep).fit(cov_type='HC1'))
    
## combining the results into a pretty stargazer format
segmented_results = Stargazer(segmented_regs)
segmented_results.custom_columns(segmented_dep.columns.tolist(), [1]*len(segmented_dep.columns))
segmented_results.show_model_numbers(False)
segmented_results.covariate_order(crisis_indep.columns.tolist())
segmented_results.significance_levels([0.05, 0.01, 0.001])
segmented_results

In [17]:
## creating a single large table describing results from all regressions above
combined_results = Stargazer(base_regs + crisis_regs + segmented_regs)
combined_results.custom_columns(2*base_dep.columns.tolist() + segmented_dep.columns.tolist(), 
                                [1]*(2*len(base_dep.columns)+len(segmented_dep.columns))
                               )
combined_results.covariate_order(crisis_indep.columns.tolist())
combined_results.significance_levels([0.05, 0.01, 0.001])
combined_results

In [18]:
## exporting the combined results to an html file for easier distribution
with open('all regression results.html', 'w') as f:
    f.write(combined_results.render_html())